## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  


In [12]:
import pandas as pd
import requests

sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape

(27028, 69)


(27028, 69)

2. Conte o número de valores *missing* por variável.  

In [13]:
sinasc.isnull().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

3. Crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

In [14]:
sinasc_reduzido = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc_reduzido.head()

sinasc_reduzido.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas dadas em 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  



In [15]:
sinasc_reduzido.dropna(subset = ['APGAR5'], inplace = True)
sinasc_reduzido.isnull().sum()

C:\Users\Artur\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64



5. Observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>


In [16]:
sinasc_reduzido.ESTCIVMAE.fillna(9, inplace = True)
sinasc_reduzido.ESCMAE.fillna(9, inplace = True)
sinasc_reduzido.isnull().sum()

C:\Users\Artur\anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE           0
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  



In [17]:
sinasc_reduzido.QTDFILVIVO.fillna(0, inplace = True)
sinasc_reduzido.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE           0
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

7. Das restantes, decida que valores te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  



In [18]:
print(sinasc_reduzido.GESTACAO.value_counts(normalize = True)*100)
sinasc_reduzido.GESTACAO.fillna('nao preenchido', inplace = True)
sinasc_reduzido.isnull().sum()

37 a 41 semanas        87.607453
32 a 36 semanas         8.689564
42 semanas e mais       2.715003
28 a 31 semanas         0.630130
22 a 27 semanas         0.330624
Menos de 22 semanas     0.027228
Name: GESTACAO, dtype: float64


LOCNASC        0
IDADEMAE       0
ESTCIVMAE      0
ESCMAE         0
QTDFILVIVO     0
GESTACAO       0
GRAVIDEZ      76
CONSULTAS      0
APGAR5         0
dtype: int64

Foi criado a opção 'não preenchido' para nao afetar a proporção das outras opções, o que poderia influenciar as análises que serão feitas.

In [19]:
print(sinasc_reduzido.GRAVIDEZ.value_counts(normalize = True)*100)
sinasc_reduzido.GRAVIDEZ.fillna('Única', inplace = True)
print(sinasc_reduzido.GRAVIDEZ.value_counts(normalize = True)*100)
sinasc_reduzido.isnull().sum()

Única              98.163805
Dupla               1.817572
Tríplice e mais     0.018623
Name: GRAVIDEZ, dtype: float64
Única              98.168988
Dupla               1.812442
Tríplice e mais     0.018570
Name: GRAVIDEZ, dtype: float64


LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

Para a variável Gravidez, foi decidido utilizar o valor 'Única' por ser a maior probabilidade de ocorrer sem prejudicar a proporção dos dados.

8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>


In [20]:
classifica_apgar = []
for i in sinasc_reduzido.APGAR5:
    if i >= 8:
        classifica_apgar.append('normal')
    elif (i >= 6) and (i < 8):
        classifica_apgar.append('asfixia leve')
    elif i >= 4 and i < 6:
        classifica_apgar.append('asfixia moderada')
    else:
        classifica_apgar.append('asfixia severa')
        
sinasc_reduzido['classificacao_a'] = classifica_apgar
sinasc_reduzido

C:\Users\Artur\AppData\Local\Temp/ipykernel_21588/382948417.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_reduzido['classificacao_a'] = classifica_apgar


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,classificacao_a
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal


9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [21]:
sinasc_reduzido.columns

sinasc_reduzido.rename(str.lower, axis='columns', inplace = True)
alterar = {'locnasc': 'loc_nasc', 'idademae' : 'idade_mae', 'estcivmae' : 'est_civ_mae', 'escmae' : 'esc_mae', 
         'qtdfilvivo': 'qtd_fil_vivo', 'gestacao' : 'gestacao',
         'gravidez' : 'gravidez', 'consultas' : 'consultas', 'apgar5' : 'apgar_5', 
         'classificacao_a': 'classificacao_a'}
sinasc_reduzido.rename(columns = alterar, inplace = True)
sinasc_reduzido

C:\Users\Artur\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar_5,classificacao_a
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal
